In [11]:
# 필요한 라이브러리 불러오기
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD


In [12]:
# 분류할 클래스의 수
num_classes = 4

# 한번에 학습시킬 이미지의 개수
batch_size = 32

# 학습 에포크의 수
epochs = 5

# 학습에 이용할 훈련 데이터와 검증 데이터의 샘플 수
num_train_samples = 400
num_val_samples = 400


In [13]:
# ResNet50 모델 불러오기
base_model = ResNet50(weights='imagenet', include_top=False)

# 새로운 fully connected layer 추가하기
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# 새로운 모델 생성하기
model = Model(inputs=base_model.input, outputs=predictions)

# base_model layer 동결하기
for layer in base_model.layers:
    layer.trainable = False

# 학습할 데이터셋 경로 설정하기
train_dir = '/content/drive/MyDrive/음식/'
val_dir = '/content/drive/MyDrive/음식/'

# 데이터셋 augmentation 및 전처리 설정하기
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

# 모델 컴파일하기
model.compile(optimizer=SGD(lr=0.001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=num_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=num_val_samples // batch_size)

Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


<ipython-input-13-e59582253b48>:48: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
12/12 [==============================] - 71s 6s/step - loss: 2.6204 - accuracy: 0.2663 - val_loss: 1.5121 - val_accuracy: 0.2656
Epoch 2/5
12/12 [==============================] - 64s 6s/step - loss: 1.4188 - accuracy: 0.2364 - val_loss: 1.3709 - val_accuracy: 0.2604
Epoch 3/5
12/12 [==============================] - 63s 6s/step - loss: 1.3783 - accuracy: 0.2772 - val_loss: 1.3537 - val_accuracy: 0.3880
Epoch 4/5
12/12 [==============================] - 65s 6s/step - loss: 1.3756 - accuracy: 0.2935 - val_loss: 1.3844 - val_accuracy: 0.2604
Epoch 5/5
12/12 [==============================] - 45s 4s/step - loss: 1.3625 - accuracy: 0.2772 - val_loss: 1.3116 - val_accuracy: 0.4271


In [14]:
#Resnet50 모델 저장
model.save('/content/drive/MyDrive/Resnet50model')

In [15]:
import numpy as np
from tensorflow.keras.preprocessing import image

# 학습한 모델 불러오기
model = tf.keras.models.load_model('/content/drive/MyDrive/Resnet50model')

# 예측할 이미지 불러오기
img_path = '/content/drive/MyDrive/테스트이미지/2/볶1.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0  # 이미지 전처리

# 이미지에 대한 예측 결과 출력하기
preds = model.predict(x)
class_indices = train_generator.class_indices
inv_map = {v: k for k, v in class_indices.items()}
pred_class = inv_map[np.argmax(preds[0])]
print("Predicted class:", pred_class)

1/1 [==============================] - 1s 1s/step
Predicted class: 보쌈
